In [2]:
import numpy as np
import plotly.express as px
import math
import plotly.figure_factory as ff
import pandas as pd

In [3]:
def simulador(lambda_= 1, mi_= 2, simulation_total_time = 10000, deterministic=False):
    n = simul_time = waiting_line_time = number_of_arrivals = system_spent_time = clients_in_system = clients_in_line = last_event_time = 0
    line_queue, arrivals = [], []

    exponential = (lambda mi: 1/mi) if deterministic else (lambda mi: np.random.exponential(scale=1/mi))

    while simul_time < simulation_total_time:
        arrival_time = np.random.exponential(scale=1/lambda_)
        departure_time = exponential(mi_)
        last_event_time = simul_time

        if n == 0 or arrival_time < departure_time:
            simul_time += arrival_time
            number_of_arrivals += 1
            clients_in_system += (simul_time - last_event_time) * n
            clients_in_line += (simul_time - last_event_time) * len(line_queue)
            n += 1
            arrivals.append(simul_time)

            if n > 1:
                line_queue.append(simul_time)

        else:
            simul_time += departure_time
            clients_in_system += (simul_time - last_event_time) * n
            clients_in_line += (simul_time - last_event_time) * len(line_queue)

            if len(line_queue) > 0:
                waiting_line_time += simul_time - line_queue.pop(0)
            n -= 1
            system_spent_time += simul_time - arrivals.pop(0)

    
    avg_waiting_time = waiting_line_time / number_of_arrivals
    avg_system_time = system_spent_time / number_of_arrivals
    avg_clients_in_system = clients_in_system / simul_time
    avg_clients_in_line = clients_in_line / simul_time

    results = {
        'people_in_line': n,
        'simul_time': simul_time,
        'avg_waiting_time': avg_waiting_time,
        'avg_system_time': avg_system_time,
        'number_of_arrivals': number_of_arrivals,
        'avg_clients_in_system': avg_clients_in_system,
        'avg_clients_in_line': avg_clients_in_line
    }

    return results

In [4]:
# obtendo dados simulados
data = simulador(lambda_= 1, mi_= 2, simulation_total_time = 100000)

In [5]:
print(f"Total de pessoas que foram ao banco na simulação: {data['number_of_arrivals']}")
print(f"Número de pessoas na fila ao final da simulação: {data['people_in_line']}")
print(f"Tempo total da simulação: {data['simul_time']:.4f}")
print(f"Tempo médio do cliente no sistema: {(data['avg_system_time']):.4f}")
print(f"Tempo médio de espera na fila: {(data['avg_waiting_time']):.4f}")
print(f"Número médio de clientes no sistema: {(data['avg_clients_in_system']):.4f}")
print(f"Número médio de clientes na fila: {(data['avg_clients_in_line']):.4f}")

Total de pessoas que foram ao banco na simulação: 99986
Número de pessoas na fila ao final da simulação: 3
Tempo total da simulação: 100000.1080
Tempo médio do cliente no sistema: 0.9963
Tempo médio de espera na fila: 0.4987
Número médio de clientes no sistema: 0.9962
Número médio de clientes na fila: 0.4986


## Visualizações e Intervalo de Confiança

Vamos verificar o intervalo de confiança para o tempo médio do cliente no sistema.

In [16]:
def get_z_score(confidence_level):
    if confidence_level == 0.9:
        return 1.645
    elif confidence_level == 0.95:
        return 1.96
    elif confidence_level == 0.99:
        return

In [17]:
def simulator_confidence_interval(lambda_, mi_, sample_size, conf_level, simulation_total_time = 1000, name='avg_waiting_time', deterministic=False):
    # coletar amostra
    results = [simulador(lambda_, mi_, simulation_total_time, deterministic)[name] for _ in range(sample_size)]

    # calcular intervalo de confiança
    x_barra = np.mean(results)
    z = get_z_score(conf_level)
    s = np.std(results, ddof=1) # ddof=1 para usar o desvio padrão amostral
    limite_inferior = x_barra - z*s/math.sqrt(sample_size)
    limite_superior = x_barra + z*s/math.sqrt(sample_size)

    print(f"Intervalo de confiança de {name} a {conf_level*100:.0f}%: \nInferior:{limite_inferior:.2f}\nSuperior: {limite_superior:.2f}")

    return (limite_inferior, limite_superior), results

In [18]:
tipo = 'avg_waiting_time'
intervalo, data = simulator_confidence_interval(lambda_= 1, mi_= 2, sample_size = 1000, conf_level = 0.95, name=tipo)

Intervalo de confiança de avg_waiting_time a 95%: 
Inferior:0.49
Superior: 0.50


In [19]:
fig = ff.create_distplot([data], ['Tempo no sistema'], curve_type='normal', bin_size=0.006)
fig.show()

In [121]:
px.box(data, orientation='h', labels={"value": tipo})

## Remoção de  outliers

In [122]:
q1 = np.quantile(data, 0.25)
q3 = np.quantile(data, 0.75)
iqr = q3 - q1

upper_bound = q3 + (1.5 * iqr)
lower_bound = q1 - (1.5 * iqr)

data_without_outliers = [x for x in data if (x >= lower_bound) and (x <= upper_bound)]

outlier_percentage = (len(data) - len(data_without_outliers)) / len(data) * 100

print(f"Porcentagem de outliers removidos: {outlier_percentage} %")


Porcentagem de outliers removidos: 1.5 %


In [123]:
fig = ff.create_distplot([data_without_outliers], ['Tempo no sistema'], curve_type='normal', bin_size=0.006)
fig.show()

### Salvando informações

In [124]:
# salvando dados para comparação
df = pd.DataFrame(data, columns=['simulador_1'])
df.to_csv('dados/simulador1.csv', index=False)

# Avaliando Resultados

Link para referência: https://people.revoledu.com/kardi/tutorial/Queuing/MM1-Queuing-System.html

$W$ = tempo médio que o cliente permanece no sistema (tempo de espera na fila + tempo de serviço). ${W = \frac{1}{\mu - \lambda}}$.\
$W_q$ = tempo médio que os clientes devem esperar na fila até serem atendidos. ${W_q = W - \frac{1}{\mu}}$.\
$L$ = número médio de clientes no sistema (na fila e sendo servidos). ${L = \lambda W}$\
$L_q$ = número médio de clientes na fila. ${L_q = \lambda W_q}$

### Caso ${\lambda} = 1$ e ${\mu} = 2$

*Tomando uma simulação como exemplo, temos a seguinte saída:*

Total de pessoas que foram ao banco na simulação: ``99388``\
Número de pessoas na fila ao final da simulação: ``1``\
Tempo total da simulação: ``100000.6943``\
Tempo médio do cliente no sistema: ``1.0001``\
Tempo médio de espera na fila: ``0.4995``\
Número médio de clientes no sistema: ``0.9940``\
Número médio de clientes na fila: ``0.4965``

Como vimos mais acima:
${W = \frac{1}{\mu - \lambda}}$. Logo, com ${\lambda = 1}$ e ${\mu = 2}$ temos: ${W = \frac{1}{2 - 1}}$ = 1\
${W_q = W - \frac{1}{\mu}}$. Logo, com ${W= 1}$ e ${\mu = 2}$ temos: ${W_q = 1 - \frac{1}{2}}$ = ${\frac{1}{2}}$\
${L = \lambda W}$. Logo, com ${W= 1}$ e ${\lambda = 1}$ temos: ${L = 1}$\
${L_q = \lambda W_q}$. Logo, com ${W_q= \frac{1}{2}}$ e ${\lambda = 1}$ temos: ${L_q = \frac{1}{2}}$

Os resultados condizem com os valores da simulação!!

In [125]:
_ = simulator_confidence_interval(lambda_= 1, mi_= 2, sample_size = 1000, conf_level = 0.95, name='avg_waiting_time')

Intervalo de confiança de avg_waiting_time a 95%: 
Inferior:0.49
Superior: 0.50


### Caso ${\lambda} = 2$ e ${\mu} = 4$

*Tomando uma simulação como exemplo, temos a seguinte saída:*

Total de pessoas que foram ao banco na simulação: ``200215``\
Número de pessoas na fila ao final da simulação: ``8``\
Tempo total da simulação: ``100000.0323``\
Tempo médio do cliente no sistema: ``0.5056``\
Tempo médio de espera na fila: ``0.2556``\
Número médio de clientes no sistema: ``1.0124``\
Número médio de clientes na fila: ``0.5117``

Como vimos mais acima:
${W = \frac{1}{\mu - \lambda}}$. Logo, com ${\lambda = 2}$ e ${\mu = 4}$ temos: ${W = \frac{1}{4 - 2}} = \frac{1}{2}$\
${W_q = W - \frac{1}{\mu}}$. Logo, com ${W= \frac{1}{2}}$ e ${\mu = 4}$ temos: ${W_q = \frac{1}{2} - \frac{1}{4}}$ = ${\frac{1}{4}}$\
${L = \lambda W}$. Logo, com ${W= \frac{1}{2}}$ e ${\lambda = 2}$ temos: ${L = 2\frac{1}{2} = 1}$\
${L_q = \lambda W_q}$. Logo, com ${W_q= \frac{1}{4}}$ e ${\lambda = 2}$ temos: ${L_q = \frac{2}{4} = \frac{1}{2}}$

Os resultados condizem com os valores da simulação!!

In [126]:
_ = simulator_confidence_interval(lambda_= 2, mi_= 4, sample_size = 1000, conf_level = 0.95, name='avg_waiting_time')

Intervalo de confiança de avg_waiting_time a 95%: 
Inferior:0.25
Superior: 0.25


### Caso ${\lambda} = 1$ e ${\mu} = 4$

*Tomando uma simulação como exemplo, temos a seguinte saída:*

Total de pessoas que foram ao banco na simulação: ``99688``\
Número de pessoas na fila ao final da simulação: ``1``\
Tempo total da simulação: ``100000.4030``\
Tempo médio do cliente no sistema: ``0.3333``\
Tempo médio de espera na fila: ``0.0829``\
Número médio de clientes no sistema: ``0.3323``\
Número médio de clientes na fila: ``0.0826``

Como vimos mais acima:
${W = \frac{1}{\mu - \lambda}}$. Logo, com ${\lambda = 1}$ e ${\mu = 4}$ temos: ${W = \frac{1}{4 - 1}} = \frac{1}{3}$\
${W_q = W - \frac{1}{\mu}}$. Logo, com ${W= \frac{1}{3}}$ e ${\mu = 4}$ temos: ${W_q = \frac{1}{3} - \frac{1}{4}}$ = ${\frac{1}{12}}$\
${L = \lambda W}$. Logo, com ${W= \frac{1}{3}}$ e ${\lambda = 1}$ temos: ${L = \frac{1}{3}}$\
${L_q = \lambda W_q}$. Logo, com ${W_q= \frac{1}{12}}$ e ${\lambda = 1}$ temos: ${L_q = \frac{1}{12}}$

Os resultados condizem com os valores da simulação!!

In [127]:
_ = simulator_confidence_interval(lambda_= 1, mi_= 4, sample_size = 1000, conf_level = 0.95, name='avg_waiting_time')

Intervalo de confiança de avg_waiting_time a 95%: 
Inferior:0.08
Superior: 0.08


### Caso ${\lambda} = 1$ e ${\mu} = 2$ Determinístico

In [128]:
data = simulador(lambda_=1, mi_= 2,simulation_total_time=100000, deterministic=True)

In [129]:
print(f"Total de pessoas que foram ao banco na simulação: {data['number_of_arrivals']}")
print(f"Número de pessoas na fila ao final da simulação: {data['people_in_line']}")
print(f"Tempo total da simulação: {data['simul_time']:.4f}")
print(f"Tempo médio do cliente no sistema: {(data['avg_clients_in_system']):.4f}")
print(f"Tempo médio de espera na fila: {(data['avg_waiting_time']):.4f}")
print(f"Número médio de clientes no sistema: {(data['avg_clients_in_system']):.4f}")
print(f"Número médio de clientes na fila: {(data['avg_clients_in_line']):.4f}")

Total de pessoas que foram ao banco na simulação: 99985
Número de pessoas na fila ao final da simulação: 1
Tempo total da simulação: 100000.9398
Tempo médio do cliente no sistema: 1.8363
Tempo médio de espera na fila: 1.1883
Número médio de clientes no sistema: 1.8363
Número médio de clientes na fila: 1.1881


In [130]:
_ = simulator_confidence_interval(lambda_= 1, mi_= 2, sample_size = 1000, conf_level = 0.95, name='avg_waiting_time', deterministic=True)

Intervalo de confiança de avg_waiting_time a 95%: 
Inferior:1.17
Superior: 1.20
